In [1]:
# imports

import asyncio
import json
import os
import matplotlib
import neptune
import nest_asyncio
import numpy as np
import pandas as pd
import time

from collections import defaultdict
from datetime import date
from itertools import product
from matplotlib import pyplot
from poke_env.environment.abstract_battle import AbstractBattle
from poke_env.player.battle_order import ForfeitBattleOrder
from poke_env.player.player import Player
from scipy.interpolate import griddata
from poke_env.player import Gen8EnvSinglePlayer, RandomPlayer
from poke_env import AccountConfiguration, ShowdownServerConfiguration
from poke_env.player import player as Sampleplayer
# from src.PlayerQLearning import Player as PlayerQLearning
import numpy as np
import asyncio
import numpy as np
from stable_baselines3 import A2C
from gymnasium.spaces import Box
from poke_env.data import GenData

from poke_env.player import Gen9EnvSinglePlayer, RandomPlayer
import os

[neptune] [warning] NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.


In [2]:
# global configs

debug = True
save_to_json_file = True
use_validation = True
use_neptune = False

nest_asyncio.apply()
np.random.seed(0)

if use_neptune:
    run = neptune.init_run(project='jukainite/pokeREL',
                       api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI2MTM2NzQ3NS0yODQ0LTRmNGItYWRmZi0yNjI1MDRiMDMxYjYifQ==',
                       tags=["Q-learning FA team2 vs team1"])


In [3]:
from poke_env.data import GenData
GEN_9_DATA = GenData.from_gen(9)

In [4]:
# Definition of agent's team (Pokémon Showdown template)

OP_TEAM = """
Iron Jugulis @ Booster Energy  
Ability: Quark Drive  
Tera Type: Steel  
EVs: 4 Atk / 252 SpA / 252 Spe  
Naive Nature  
- Knock Off  
- Hurricane  
- Earth Power  
- Taunt  

Roaring Moon @ Booster Energy  
Ability: Protosynthesis  
Tera Type: Flying  
EVs: 252 Atk / 4 Def / 252 Spe  
Jolly Nature  
- Dragon Dance  
- Acrobatics  
- Knock Off  
- Taunt    

TWINKATON (Tinkaton) @ Air Balloon  
Ability: Pickpocket  
Tera Type: Water  
EVs: 248 HP / 24 SpD / 236 Spe  
Jolly Nature  
- Stealth Rock  
- Play Rough  
- Thunder Wave  
- Encore  

Iron Moth @ Booster Energy  
Ability: Quark Drive  
Tera Type: Fairy  
EVs: 124 Def / 132 SpA / 252 Spe  
Timid Nature  
IVs: 0 Atk  
- Fiery Dance  
- Sludge Wave  
- Psychic  
- Dazzling Gleam  

Great Tusk @ Booster Energy  
Ability: Protosynthesis  
Tera Type: Poison  
EVs: 252 HP / 4 Def / 252 Spe  
Jolly Nature  
- Headlong Rush  
- Ice Spinner  
- Bulk Up  
- Rapid Spin  

Dragapult @ Choice Specs  
Ability: Clear Body  
Tera Type: Ghost  
EVs: 252 SpA / 4 SpD / 252 Spe  
Timid Nature  
- Draco Meteor  
- Shadow Ball  
- Thunderbolt  
- U-turn  

"""


In [5]:
# Definition of opponent's team (Pokémon Showdown template)

OUR_TEAM= """
Daenerys (Kingambit) (F) @ Leftovers  
Ability: Supreme Overlord  
Tera Type: Dark  
EVs: 160 HP / 252 Atk / 96 Spe  
Adamant Nature  
- Kowtow Cleave  
- Iron Head  
- Sucker Punch  
- Swords Dance  

Kristine (Cinderace) (F) @ Heavy-Duty Boots  
Ability: Blaze  
Shiny: Yes  
Tera Type: Flying  
EVs: 144 HP / 112 Atk / 252 Spe  
Jolly Nature  
- Pyro Ball  
- Will-O-Wisp  
- Court Change  
- U-turn  

Homelandor (Landorus-Therian) @ Rocky Helmet  
Ability: Intimidate  
Shiny: Yes  
Tera Type: Dragon  
EVs: 248 HP / 244 Def / 16 Spe  
Bold Nature  
- Earth Power  
- Taunt  
- Stealth Rock  
- U-turn  

WALL-Y (Iron Valiant) @ Booster Energy  
Ability: Quark Drive  
Tera Type: Ghost  
EVs: 176 Atk / 80 SpA / 252 Spe  
Naive Nature  
- Moonblast  
- Close Combat  
- Knock Off  
- Encore  

Mr. Freeze (Kyurem) @ Choice Specs  
Ability: Pressure  
Shiny: Yes  
Tera Type: Ice  
EVs: 4 Def / 252 SpA / 252 Spe  
Timid Nature  
IVs: 0 Atk  
- Draco Meteor  
- Freeze-Dry  
- Earth Power  
- Blizzard  

SlodogChillionaire (Slowking-Galar) (M) @ Heavy-Duty Boots  
Ability: Regenerator  
Shiny: Yes  
Tera Type: Water  
EVs: 248 HP / 8 Def / 252 SpD  
Sassy Nature  
IVs: 0 Atk / 0 Spe  
- Toxic  
- Future Sight  
- Surf  
- Chilly Reception  
"""


In [6]:
NAME_TO_ID_DICT = NAME_TO_ID = {"kingambit": 0,
                                "cinderace": 1,
                                "landorustherian": 2,
                                "ironvaliant": 3,
                                "kyurem": 4,
                                "slowkinggalar": 5,
                                "ironjugulis": 0,
                                "roaringmoon": 1,
                                "tinkaton": 2,
                                "ironmoth": 3,
                                "greattusk": 4,
                                "dragapult": 5, }

In [7]:
GEN_9_DATA = GenData.from_gen(9)
rewards_a = []
win_a = []

In [8]:
class RLPlayer(Gen9EnvSinglePlayer):



    def embed_battle(self, battle):
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                    move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=GEN_9_DATA.type_chart

                )

        # We count how many pokemons have not fainted in each team
        n_fainted_mon_team = (
            len([mon for mon in battle.team.values() if mon.fainted])
        )
        n_fainted_mon_opponent = (
            len([mon for mon in battle.opponent_team.values() if mon.fainted])
        )

        state = np.concatenate([
            [NAME_TO_ID_DICT[str(battle.active_pokemon).split(' ')[0]]],
            [NAME_TO_ID_DICT[str(battle.opponent_active_pokemon).split(' ')[0]]],
            [move_base_power for move_base_power in moves_base_power],
            [move_dmg_multiplier for move_dmg_multiplier in moves_dmg_multiplier],
            [n_fainted_mon_team,
             n_fainted_mon_opponent]])

        return state

    # Computing rewards

    def reward_computing_helper(
            self,
            battle: AbstractBattle,
            *,
            fainted_value: float = 0.15,
            hp_value: float = 0.15,
            number_of_pokemons: int = 6,
            starting_value: float = 0.0,
            status_value: float = 0.15,
            victory_value: float = 1.0
    ) -> float:
        # 1st compute
        if battle not in self._reward_buffer:
            self._reward_buffer[battle] = starting_value
        current_value = 0

        # Verify if pokemon have fainted or have status
        for mon in battle.team.values():
            current_value += mon.current_hp_fraction * hp_value
            if mon.fainted:
                current_value -= fainted_value
            elif mon.status is not None:
                current_value -= status_value

        current_value += (number_of_pokemons - len(battle.team)) * hp_value

        # Verify if opponent pokemon have fainted or have status
        for mon in battle.opponent_team.values():
            current_value -= mon.current_hp_fraction * hp_value
            if mon.fainted:
                current_value += fainted_value
            elif mon.status is not None:
                current_value += status_value

        current_value -= (number_of_pokemons - len(battle.opponent_team)) * hp_value

        # Verify if we won or lost
        if battle.won:
            current_value += victory_value
        elif battle.lost:
            current_value -= victory_value

        # Value to return
        to_return = current_value - self._reward_buffer[battle]
        self._reward_buffer[battle] = current_value
        
        return to_return

    # Calling reward_computing_helper
    def calc_reward(self, last_battle, battle) -> float:
        return self.reward_computing_helper(battle, fainted_value=2, hp_value=1, victory_value=15)

    def _battle_finished_callback(self, battle):
        self.num_battles += 1
        

        self._observations[battle].put(self.embed_battle(battle))

    def describe_embedding(self):
        low = [-1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0]
        high = [3, 3, 3, 3, 3, 4, 4, 4, 4, 1, 1, 1]
        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32,
        )

In [9]:
class MaxDamagePlayer(RandomPlayer):
    def choose_move(self, battle):
        # If the player can attack, it will
        if battle.available_moves:
            # Finds the best move among available ones
            best_move = max(battle.available_moves, key=lambda move: move.base_power)
            return self.create_order(best_move)

        # If no attack is available, a random switch will be made
        else:
            return self.choose_random_move(battle)

In [10]:
opponent = MaxDamagePlayer(battle_format="gen9ou", team=OP_TEAM)
env_player = RLPlayer(battle_format="gen9ou", team=OUR_TEAM, opponent=opponent)
model = A2C('MlpPolicy', env=env_player, verbose=1)
model.load("model_%d" % 300000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


d:\REL-Project\.venv\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


In [11]:
NB_EVALUATION_EPISODES =10

In [13]:
env_player._opponent = opponent
try:
    env_player.reset_battles()
except:
    pass

for _ in range(NB_EVALUATION_EPISODES):
    done = False
    obs, info = env_player.reset()
    while not done:
        try:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, _, info = env_player.step(action)
        except:
            done = True


print(
    "DQN Evaluation: %d victories out of %d games"
    % (env_player.n_won_battles, NB_EVALUATION_EPISODES)
)
try:
    env_player.reset_battles()
except:
    pass

DQN Evaluation: 1 victories out of 10 games
